# ❄️ Snowflake Intelligence Hands-On Lab ❄️

In this session, you will learn how to:
- Perform semantic modeling to enable text-to-sql conversion via Cortex Analyst
- Use Cortex Search to derive insights from unstructured data
- Build Cortex Agents that can combine both Cortex Analyst and Cortex Search to answer questions
- Use Snowflake Intelligence as a UI layer for Cortex Agents
- Investigate Snowflake's built in observability for the agent


## 1. Data Exploration 📊
Your lab environment has come preloaded with a number of tables already populated. Let's quickly explore them to understand what kind of data we're working with.

In [ ]:
USE SF_AI_DEMO.DEMO_SCHEMA;
USE ROLE ATTENDEE_ROLE;
SHOW TABLES;

In [ ]:
-- Query Sales by Product
SELECT
    p.product_name,
    SUM(s.amount) AS total_sales_amount,
    SUM(s.units) AS total_units_sold
FROM
    sales_fact AS s
INNER JOIN
    product_dim AS p 
ON
    s.product_key = p.product_key
GROUP BY p.product_name
ORDER BY total_sales_amount DESC
LIMIT 5;

In [ ]:
-- Query total transaction amount by third party vendor
SELECT
    a.account_type,
    v.vendor_name,
    SUM(f.amount) AS total_amount_transacted
FROM
    finance_transactions AS f
INNER JOIN
    account_dim AS a ON f.account_key = a.account_key
INNER JOIN
    vendor_dim AS v ON f.vendor_key = v.vendor_key
GROUP BY 1, 2
ORDER BY total_amount_transacted DESC
LIMIT 5;

## 2 - Semantic Modeling 📈
Semantic Views are schema-level objects in Snowflake that enable us to define business metrics, entities, and their relationships. This context will be leveraged by Cortex Analyst for generating SQL to answer natural language questions.

Semantic Views are comprised of the following components:

- **Tables**: logical tables that map to Snowflake tables or views
- **Facts**: row-level attributes tied to a logical table that represent specific business events or transactions
- **Dimensions**: categorical attributes tied to a logical table that gives meaning to metrics by grouping data into meaningful categories
- **Metrics**: quantifiable measures of business performance calculated by aggregating facts or other columns from the same table
- **Named filters**: logic to filter a logical table based on some business rule
- **Relationships**: how logical tables are mapped to one another. This enables Cortex Analyst to join multiple logical tables together

Semantic Views can be created through SQL (as seen below), but also through the Snowsight UI which is what we'll walk through together in this lab.

In [ ]:

CREATE OR REPLACE SEMANTIC VIEW SF_AI_DEMO.DEMO_SCHEMA.SALES_SEMANTIC_VIEW
	tables (
		CUSTOMERS as CUSTOMER_DIM primary key (CUSTOMER_KEY) with synonyms=('clients','customers','accounts') comment='Customer information for sales analysis',
		PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('products','items','SKUs') comment='Product catalog for sales analysis',
		PRODUCT_CATEGORY_DIM primary key (CATEGORY_KEY),
		REGIONS as REGION_DIM primary key (REGION_KEY) with synonyms=('territories','regions','areas') comment='Regional information for territory analysis',
		SALES as SALES_FACT primary key (SALE_ID) with synonyms=('sales transactions','sales data') comment='All sales transactions and deals',
		SALES_REPS as SALES_REP_DIM primary key (SALES_REP_KEY) with synonyms=('sales representatives','reps','salespeople') comment='Sales representative information',
		VENDORS as VENDOR_DIM primary key (VENDOR_KEY) with synonyms=('suppliers','vendors') comment='Vendor information for supply chain analysis'
	)
	relationships (
		PRODUCT_TO_CATEGORY as PRODUCTS(CATEGORY_KEY) references PRODUCT_CATEGORY_DIM(CATEGORY_KEY),
		SALES_TO_CUSTOMERS as SALES(CUSTOMER_KEY) references CUSTOMERS(CUSTOMER_KEY),
		SALES_TO_PRODUCTS as SALES(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
		SALES_TO_REGIONS as SALES(REGION_KEY) references REGIONS(REGION_KEY),
		SALES_TO_REPS as SALES(SALES_REP_KEY) references SALES_REPS(SALES_REP_KEY),
		SALES_TO_VENDORS as SALES(VENDOR_KEY) references VENDORS(VENDOR_KEY)
	)
	facts (
		SALES.SALE_AMOUNT as amount comment='Sale amount in dollars',
		SALES.SALE_RECORD as 1 comment='Count of sales transactions',
		SALES.UNITS_SOLD as units comment='Number of units sold'
	)
	dimensions (
		CUSTOMERS.CUSTOMER_INDUSTRY as INDUSTRY with synonyms=('industry','customer type') comment='Customer industry',
		CUSTOMERS.CUSTOMER_NAME as customer_name with synonyms=('customer','client','account') comment='Name of the customer',
		PRODUCTS.PRODUCT_NAME as product_name with synonyms=('product','item') comment='Name of the product',
		PRODUCT_CATEGORY_DIM.CATEGORY_NAME as CATEGORY_NAME with synonyms=('category','product category') comment='The category to which a product belongs',
		PRODUCT_CATEGORY_DIM.VERTICAL as VERTICAL with synonyms=('industry','sector','market') comment='The industry or sector in which a product is categorized',
		REGIONS.REGION_NAME as region_name with synonyms=('region','territory','area') comment='Name of the region',
		SALES.SALE_DATE as date with synonyms=('date','sale date','transaction date') comment='Date of the sale',
		SALES.SALE_MONTH as MONTH(date) comment='Month of the sale',
		SALES.SALE_YEAR as YEAR(date) comment='Year of the sale',
		SALES_REPS.SALES_REP_NAME as REP_NAME with synonyms=('sales rep','representative','salesperson') comment='Name of the sales representative',
		VENDORS.VENDOR_NAME as vendor_name with synonyms=('vendor','supplier','provider') comment='Name of the vendor'
	)
	metrics (
		SALES.AVERAGE_DEAL_SIZE as AVG(sales.amount) comment='Average deal size',
		SALES.AVERAGE_UNITS_PER_SALE as AVG(sales.units) comment='Average units per sale',
		SALES.TOTAL_DEALS as COUNT(sales.sale_record) comment='Total number of deals',
		SALES.TOTAL_REVENUE as SUM(sales.amount) comment='Total sales revenue',
		SALES.TOTAL_UNITS as SUM(sales.units) comment='Total units sold'
	)
	comment='Semantic view for sales analysis and performance tracking';
     


In [ ]:
-- We'll create this together in the Snowsight UI!
CREATE OR REPLACE SEMANTIC VIEW SF_AI_DEMO.DEMO_SCHEMA.FINANCE_SEMANTIC_VIEW
    tables (
        TRANSACTIONS as FINANCE_TRANSACTIONS primary key (TRANSACTION_ID) with synonyms=('finance transactions','financial data') comment='All financial transactions across departments',
        ACCOUNTS as ACCOUNT_DIM primary key (ACCOUNT_KEY) with synonyms=('chart of accounts','account types') comment='Account dimension for financial categorization',
        DEPARTMENTS as DEPARTMENT_DIM primary key (DEPARTMENT_KEY) with synonyms=('business units','departments') comment='Department dimension for cost center analysis',
        VENDORS as VENDOR_DIM primary key (VENDOR_KEY) with synonyms=('suppliers','vendors') comment='Vendor information for spend analysis',
        PRODUCTS as PRODUCT_DIM primary key (PRODUCT_KEY) with synonyms=('products','items') comment='Product dimension for transaction analysis',
        CUSTOMERS as CUSTOMER_DIM primary key (CUSTOMER_KEY) with synonyms=('clients','customers') comment='Customer dimension for revenue analysis'
    )
    relationships (
        TRANSACTIONS_TO_ACCOUNTS as TRANSACTIONS(ACCOUNT_KEY) references ACCOUNTS(ACCOUNT_KEY),
        TRANSACTIONS_TO_DEPARTMENTS as TRANSACTIONS(DEPARTMENT_KEY) references DEPARTMENTS(DEPARTMENT_KEY),
        TRANSACTIONS_TO_VENDORS as TRANSACTIONS(VENDOR_KEY) references VENDORS(VENDOR_KEY),
        TRANSACTIONS_TO_PRODUCTS as TRANSACTIONS(PRODUCT_KEY) references PRODUCTS(PRODUCT_KEY),
        TRANSACTIONS_TO_CUSTOMERS as TRANSACTIONS(CUSTOMER_KEY) references CUSTOMERS(CUSTOMER_KEY)
    )
    facts (
        TRANSACTIONS.TRANSACTION_AMOUNT as amount comment='Transaction amount in dollars',
        TRANSACTIONS.TRANSACTION_RECORD as 1 comment='Count of transactions'
    )
    dimensions (
        TRANSACTIONS.TRANSACTION_DATE as date with synonyms=('date','transaction date') comment='Date of the financial transaction',
        TRANSACTIONS.TRANSACTION_MONTH as MONTH(date) comment='Month of the transaction',
        TRANSACTIONS.TRANSACTION_YEAR as YEAR(date) comment='Year of the transaction',
        ACCOUNTS.ACCOUNT_NAME as account_name with synonyms=('account','account type') comment='Name of the account',
        ACCOUNTS.ACCOUNT_TYPE as account_type with synonyms=('type','category') comment='Type of account (Income/Expense)',
        DEPARTMENTS.DEPARTMENT_NAME as department_name with synonyms=('department','business unit') comment='Name of the department',
        VENDORS.VENDOR_NAME as vendor_name with synonyms=('vendor','supplier') comment='Name of the vendor',
        PRODUCTS.PRODUCT_NAME as product_name with synonyms=('product','item') comment='Name of the product',
        CUSTOMERS.CUSTOMER_NAME as customer_name with synonyms=('customer','client') comment='Name of the customer'
    )
    metrics (
        TRANSACTIONS.AVERAGE_AMOUNT as AVG(transactions.amount) comment='Average transaction amount',
        TRANSACTIONS.TOTAL_AMOUNT as SUM(transactions.amount) comment='Total transaction amount',
        TRANSACTIONS.TOTAL_TRANSACTIONS as COUNT(transactions.transaction_record) comment='Total number of transactions'
    )
    comment='Semantic view for financial analysis and reporting';

## 3. Cortex Agents 🤖
Now that we have Cortex Analyst set up to generate SQL for answering questions about our structured data, we can now start to use that from a Cortex Agent.

Cortex Agents are the brains of this demo. They configured with specific tools like Cortex Analyst, Cortex Search, Custom Tools, Cortex Knowledge Extensions (CKEs), etc.. to answer questions and carry out tasks on behalf of users. When receiving a request, the Agent will go through an iterative process of building out a plan to complete the request, executing tools to retrieve data or take action, and then reflecting on its progress and repeating as needed.

In [ ]:
-- Create the first version of our Company Chatbot Agent
USE ROLE ATTENDEE_ROLE;

CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Company_Chatbot_Agent_Structured
WITH PROFILE='{"display_name": "1-Company Chatbot Agent - Structured Data"}'
COMMENT=$$ This is an agent that can answer questions about company specific Sales & Finance questions. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": "auto"
  },
  "instructions": {
    "response": "You are a data analyst who has access to our finance and sales datamart. If user does not specify a date range assume it for year 2025. Provide visualizations if possible. Trendlines should default to linecharts, Categories Barchart.",
    "orchestration": "Use the Cortex Analyst tools to retrieve data from the Sales and Finance datamarts when applicable.",
    "sample_questions": [
      {
        "question": "Show me monthly sales trends for 2025 with visualizations. Which months had the highest revenue?"
      },
      {
        "question": "What are our top 5 product categories by revenue in 2025?"
      },
      {
        "question": "Who are our top performing sales representatives? Show their individual revenue contributions."
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Finance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Sales Datamart",
        "description": "Allows users to query Sales data for a company in terms of Sales data such as products, sales reps & etc. "
      }
    },
  ],
  "tool_resources": {
    "Query Finance Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.FINANCE_SEMANTIC_VIEW"
    },
    "Query Sales Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.SALES_SEMANTIC_VIEW"
    },
  }
}
$$;

## Ask some sample questions!

Here are a few thought starters, but think of what else is possible using this agent through Snowflake Intelligence:
- Show me monthly sales trends for 2025 with visualizations. Which months had the highest revenue?
- What are our top 5 products by revenue in 2025? Show me their performance by region.
- Who are our top performing sales representatives? Show their individual revenue contributions.

## 4. Cortex Search for Unstructured Data 🗂️ 

In this section, will tie in the unstructured data that was processed in the Openflow portion of the lab to our Cortex Agent so that we can use it as additional context for answering questions.

To do this, we will use **Cortex Search**- a feature in Snowflake for performing low-latency, high-quality search over your data in Snowflake. Cortex Search performs "hybrid" search, which is a combination of both vector and keyword search to get the most relevant information for the task at hand. 

Cortex Search will also handle all of the complexity of managing a hybrid search service, such as the:
- embedding pipeline
- serving and storage infrastructure
- search parameter tuning
- index refreshes

This means that you can get high-quality search results through only a few simple SQL statements in Snowflake.

In [ ]:
USE SF_AI_DEMO.DEMO_SCHEMA;

-- Validate the parsed content is present from the Openflow demo
select * from parsed_content;


In [ ]:
-- Create search service for finance documents
USE SF_AI_DEMO.DEMO_SCHEMA;

CREATE OR REPLACE CORTEX SEARCH SERVICE Search_finance_docs
    ON content
    ATTRIBUTES relative_path, file_url, title
    WAREHOUSE = SNOW_INTELLIGENCE_DEMO_WH
    TARGET_LAG = '30 day'
    EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
    AS (
        SELECT
            relative_path,
            file_url,
            REGEXP_SUBSTR(relative_path, '[^/]+$') as title,
            content
        FROM parsed_content
        WHERE relative_path ILIKE 'unstructured_docs/%.pdf'
    );


### Custom Tools ⚒️
We can also add custom tools in the form of stored procedures and user defined functions (UDFs) to our Cortex Agents.

Below, we will create a tool for generating a URL for users to download any files that Snowflake Intelligence references in its results.

In [ ]:
-- Create stored procedure to generate presigned URLs for files in internal stages
-- This will enable users to download files referenced from answers in Snowflake Intelligence
CREATE OR REPLACE PROCEDURE Get_File_Presigned_URL_SP(
    RELATIVE_PATH STRING, 
    EXPIRATION_MINS INTEGER DEFAULT 60
)
RETURNS STRING
LANGUAGE SQL
COMMENT = 'Generates a presigned URL for a file in the static @INTERNAL_DATA_STAGE. Input is the RELATIVE_PATH.'
EXECUTE AS CALLER
AS
$$
DECLARE
    presigned_url STRING;
    sql_stmt STRING;
    expiration_seconds INTEGER;
    stage_name STRING DEFAULT '@SF_AI_DEMO.DEMO_SCHEMA.INTERNAL_DATA_STAGE';
BEGIN
    expiration_seconds := EXPIRATION_MINS * 60;

    sql_stmt := 'SELECT GET_PRESIGNED_URL(' || stage_name || ', ' || '''' || RELATIVE_PATH || '''' || ', ' || expiration_seconds || ') AS url';
    
    EXECUTE IMMEDIATE :sql_stmt;
    
    SELECT "URL"
    INTO :presigned_url
    FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()));
    
    RETURN :presigned_url;
END;
$$;
     

### Adding our Cortex Search service and custom tool to our Agent!

Now that we have our Cortex Search service and custom Doc URL stored proc created, we can add those as additional tools to our agent, providing additional context and functionality.

In [ ]:
-- Create the first version of our Company Chatbot Agent
USE ROLE ATTENDEE_ROLE;

CREATE OR REPLACE AGENT SNOWFLAKE_INTELLIGENCE.AGENTS.Company_Chatbot_Agent_All_Data
WITH PROFILE='{"display_name": "2-Company Chatbot Agent - Structured & Unstructured Data"}'
COMMENT=$$ This is an agent that can answer questions about company specific Sales & Finance questions. $$
FROM SPECIFICATION $$
{
  "models": {
    "orchestration": "auto"
  },
  "instructions": {
    "response": "You are a data analyst who has access to our finance and sales datamart as well as finance documents. Use the tools at your disposal to generate an answer for the user's query. If user does not specify a date range assume it for year 2025. Provide visualizations if possible. Trendlines should default to linecharts, Categories Barchart.",
    "orchestration": "Use the Cortex Analyst tools to retrieve data from the Sales and Finance datamarts when applicable. Use the Search Internal Documents: Finance tool powered by Cortex Search to retrieve any data from our internal Finance documents. Any time we are providing a document from the Search Internal Documents: Finance tool, be sure to invoke the `Dynamic_Doc_URL_Tool` so that the user can download the referenced file. Summarize all data retrieved into a concise response.",
    "sample_questions": [
      {
        "question": "What are our top 5 vendors in the last 5 years? Check our vendor management policy - are we following procurement guidelines for all transactions?"
      },
      {
        "question": "Summarize our latest financial report"
      },
      {
        "question": "Who's our preferred technology partner? What does our current contract with them look like?"
      }
    ]
  },
  "tools": [
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Finance Datamart",
        "description": "Allows users to query finance data for a company in terms of revenue & expenses."
      }
    },
    {
      "tool_spec": {
        "type": "cortex_analyst_text_to_sql",
        "name": "Query Sales Datamart",
        "description": "Allows users to query Sales data for a company in terms of Sales data such as products, sales reps & etc. "
      }
    },
    {
      "tool_spec": {
        "type": "cortex_search",
        "name": "Search Internal Documents: Finance",
        "description": "Allows users to query documents around finance topics such as vendor management, financial reports, etc.."
      }
    },
    {
      "tool_spec": {
        "type": "generic",
        "name": "Dynamic_Doc_URL_Tool",
        "description": "This tools uses the relative_path Column coming from Cortex Search tools for reference docs and returns a temp URL for users to view & download the docs.\n\nReturned URL should be presented as a HTML Hyperlink where doc title should be the text and out of this tool should be the url.\n\nURL format for PDF docs that are are like this which has no PDF in the url. Create the Hyperlink format so the PDF doc opens up in a browser instead of downloading the file.\nhttps://domain/path/unique_guid",
        "input_schema": {
          "type": "object",
          "properties": {
            "relative_path": {
              "description": "This is the RELATIVE_PATH Column value Coming from Cortex Search tool.",
              "type": "string"
            },
            "expiration_mins": {
              "description": "default should be 5",
              "type": "number"
            }
          },
          "required": [
            "relative_path",
            "expiration_mins"
          ]
        }
      }
    }
  ],
  "tool_resources": {
    "Query Finance Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.FINANCE_SEMANTIC_VIEW"
    },
    "Query Sales Datamart": {
      "semantic_view": "SF_AI_DEMO.DEMO_SCHEMA.SALES_SEMANTIC_VIEW"
    },
    "Search Internal Documents: Finance": {
      "id_column": "FILE_URL",
      "max_results": 5,
      "name": "SF_AI_DEMO.DEMO_SCHEMA.SEARCH_FINANCE_DOCS",
      "title_column": "TITLE"
    },
    "Dynamic_Doc_URL_Tool": {
      "execution_environment": {
        "query_timeout": 0,
        "type": "warehouse",
        "warehouse": "SNOW_INTELLIGENCE_DEMO_WH"
      },
      "identifier": "SF_AI_DEMO.DEMO_SCHEMA.GET_FILE_PRESIGNED_URL_SP",
      "name": "GET_FILE_PRESIGNED_URL_SP(VARCHAR, DEFAULT NUMBER)",
      "type": "procedure"
    },
  }
}
$$;

## Ask some more questions!
- What are our top 5 vendors in the last 5 years? Check our vendor management policy - are we following procurement guidelines for all transactions?
- Summarize our latest financial report
- Who's our preferred technology partner? What does our current contract with them look like?


## 5. AI Observability 🔎

After asking a few questions to our agent, we can explore Snowflake's built in AI Observability tooling to understand what tools were invoked to accomplish the given tasks.

Start by clicking on 'AI & ML' on the left hand havbar, and then opening 'Cortex Agents' in a new tab. Once on the Cortex Agents screen, choose the 'Risk_Analysis_Agent_Tools' agent. Here, you can see all of the configuration for our agent, including which tools it has available.

Next, click on 'Monitoring' and then choose one of your conversations. This is where we can see all the detail going on behind the scenes. Find a task that used Cortex Analyst - there we can see the exact SQL that was generated and executed to answer the given question. We can see all tool executions here including the input and output which greatly helps in any debugging activities.